In [8]:
from utils import read_video, save_video
from trackers import Tracker
import cv2

In [7]:
# Read Video
video_path = 'input_video/08fd33_4.mp4'
output_dir = 'output_video'

In [3]:
video_frames = read_video(video_path)

In [5]:
# 初始化Tracker
tracker = Tracker('models/best.pt')
tracks = tracker.get_object_tracks(video_frames,
                                    read_from_stub=True,
                                    stub_path='stub/track_stub.pkl')
# 画出输出
## 画出对象轨迹
output_video_frames = tracker.draw_annotation(video_frames, tracks)

In [9]:
# 保存每个队员的截图
for id, player in tracks["player"][0].items():
    bbox = player["bbox"]
    frame = video_frames[0]
    cropped = frame[int(bbox[1]) : int(bbox[3]), int(bbox[0]) : int(bbox[2])]
    cv2.imwrite(f"{output_dir}/player_{id}.jpg", cropped)

In [6]:
# Save Video
save_video(output_video_frames, f'{output_dir}/output_video.avi')

Saving Video: 100%|██████████| 750/750 [00:10<00:00, 71.79it/s]

Video saved at output_video/output_video.avi
